In [70]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from statsmodels.miscmodels.ordinal_model import OrderedModel

## Bank Churn

In [2]:
bank_df = pd.read_csv('BANK.xls - Sheet1.csv',skiprows=2)

In [3]:
len(bank_df)

248

In [4]:
bank_df.drop(0,inplace=True)

In [5]:
bank_df.drop(bank_df.tail(2).index,inplace=True)

In [6]:
bank_df.columns

Index(['Unnamed: 0', 'O Loyal', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Pleasant Ambiance', 'Comfortable seating arrangement',
       'Immediate attenttion', 'Good Response on Phone',
       'Errors in Passbook entries', 'Time to issue cheque book',
       'Time to sanction loan', 'Time to clear outstation cheques',
       'Issue of clean currency notes', 'Facility to pay bills',
       'Distance to residence', 'Distance to workplace',
       'Courteous staff behaviour', 'Enough parking place'],
      dtype='object')

In [7]:
bank_df.rename(columns={'Unnamed: 0':'Sr. No','O Loyal':'Response','Unnamed: 2':'Branch','Unnamed: 3':'Occupation','Unnamed: 4':'Age','Unnamed: 5':'Gender'},inplace=True)

In [8]:
bank_df.set_index('Sr. No',inplace=True)

In [9]:
bank_df.head()

,Response,Branch,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place
Sr. No,,,,,,,,,,,,,,,,,,,
1,0,B1,6,5,2,9,5,8,6,8,0,6,4,6,3,10,9,8,-2
2,0,B1,1,5,1,10,8,8,6,10,8,9,6,8,6,8,7,10,10
3,0,B1,0,5,1,4,8,7,8,5,7,4,5,7,6,8,7,9,6
4,0,B1,6,4,2,4,6,6,8,6,4,4,5,6,6,9,6,9,5
5,0,B1,1,2,1,10,8,6,10,8,8,5,7,8,7,10,10,5,8


In [10]:
bank_df.tail()

,Response,Branch,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place
Sr. No,,,,,,,,,,,,,,,,,,,
241,1,B6,2,2,1,10,10,9,10,9,10,9,10,8,10,10,9,10,9
242,1,B6,1,3,1,9,8,9,9,4,8,6,7,8,9,10,9,9,8
243,1,B6,1,3,1,8,8,5,6,9,9,8,9,7,7,6,6,10,9
244,1,B6,3,4,*,7,6,5,7,5,7,6,7,7,6,7,7,6,5
245,1,B6,6,3,2,7,5,7,6,5,-1,5,7,5,5,6,5,5,5


In [11]:
bank_df[bank_df.duplicated()]

,Response,Branch,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place
Sr. No,,,,,,,,,,,,,,,,,,,


In [12]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245 entries, 1 to 245
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Response                          245 non-null    object
 1   Branch                            245 non-null    object
 2   Occupation                        245 non-null    object
 3   Age                               245 non-null    object
 4   Gender                            245 non-null    object
 5   Pleasant Ambiance                 245 non-null    object
 6   Comfortable seating arrangement   245 non-null    object
 7   Immediate attenttion              245 non-null    object
 8   Good Response on Phone            245 non-null    object
 9   Errors in Passbook entries        245 non-null    object
 10  Time to issue cheque book         245 non-null    object
 11  Time to sanction loan             245 non-null    object
 12  Time to clear outstation ch

##### Columns with non numeric values

In [13]:
cols_with_non_numeric = [i for i in bank_df.columns if all(bank_df[i].str.match('^-?\d')) == False]

In [14]:
cols_with_non_numeric

['Branch', 'Occupation', 'Gender']

In [15]:
bank_df[bank_df['Gender'].str.match('\d') == False]

,Response,Branch,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place
Sr. No,,,,,,,,,,,,,,,,,,,
39,0,B2,2,3,na,6,5,6,6,6,8,4,5,5,6,5,6,7,5
84,0,B5,0,4,*,6,4,5,8,6,8,8,7,5,4,9,10,7,8
162,0,B8,0,5,*,3,5,6,9,8,8,4,2,7,9,9,8,3,7
169,0,B9,1,5,*,9,10,5,6,8,5,10,6,7,6,5,8,7,6
244,1,B6,3,4,*,7,6,5,7,5,7,6,7,7,6,7,7,6,5


In [16]:
bank_df.drop(bank_df[bank_df['Gender'].str.match('\d') == False].index,inplace=True)

In [17]:
bank_df[bank_df['Occupation'].str.match('\d') == False]

,Response,Branch,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place
Sr. No,,,,,,,,,,,,,,,,,,,
36,0,B2,na,3,1,6,7,6,9,5,7,6,5,5,8,5,8,6,5


In [18]:
bank_df.drop(bank_df[bank_df['Occupation'].str.match('\d') == False].index,inplace=True)

##### Columns with negative values for score

In [19]:
all(bank_df['Pleasant Ambiance'].str.match('^-\d'))

False

In [20]:
cols_with_neg_numeric = [i for i in bank_df.columns if len(bank_df[bank_df[i].str.match('^-\d') == False]) != len(bank_df)]

In [21]:
cols_with_neg_numeric

['Pleasant Ambiance',
 'Comfortable seating arrangement',
 'Immediate attenttion',
 'Good Response on Phone',
 'Errors in Passbook entries',
 'Time to issue cheque book',
 'Time to sanction loan',
 'Time to clear outstation cheques',
 'Issue of clean currency notes',
 'Facility to pay bills',
 'Distance to residence',
 'Distance to workplace',
 'Courteous staff behaviour',
 'Enough parking place']

In [22]:
bank_df.columns.drop(['Branch'])

Index(['Response', 'Occupation', 'Age', 'Gender', 'Pleasant Ambiance',
       'Comfortable seating arrangement', 'Immediate attenttion',
       'Good Response on Phone', 'Errors in Passbook entries',
       'Time to issue cheque book', 'Time to sanction loan',
       'Time to clear outstation cheques', 'Issue of clean currency notes',
       'Facility to pay bills', 'Distance to residence',
       'Distance to workplace', 'Courteous staff behaviour',
       'Enough parking place'],
      dtype='object')

In [23]:
bank_df[bank_df.columns.drop(['Branch'])].values

array([['0', '6', '5', ..., '9', '8', '-2'],
       ['0', '1', '5', ..., '7', '10', '10'],
       ['0', '0', '5', ..., '7', '9', '6'],
       ...,
       ['1', '1', '3', ..., '9', '9', '8'],
       ['1', '1', '3', ..., '6', '10', '9'],
       ['1', '6', '3', ..., '5', '5', '5']], dtype=object)

In [24]:
rows_with_non_response = list(np.where(np.any((bank_df[bank_df.columns.drop(['Branch'])].astype(int).values < 0) == True,axis=1)))

In [25]:
bank_df[(bank_df[bank_df.columns.drop(['Branch'])].astype(int).values < 0)]

,Response,Branch,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place
Sr. No,,,,,,,,,,,,,,,,,,,
1,0,B1,6,5,2,9,5,8,6,8,0,6,4,6,3,10,9,8,-2
17,0,B1,6,4,2,8,5,-1,7,5,8,4,4,6,5,6,5,9,3
18,0,B1,0,5,1,7,6,-3,7,-3,-3,8,7,-3,7,8,-3,8,8
18,0,B1,0,5,1,7,6,-3,7,-3,-3,8,7,-3,7,8,-3,8,8
18,0,B1,0,5,1,7,6,-3,7,-3,-3,8,7,-3,7,8,-3,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,1,B2,1,4,2,8,5,10,7,4,6,-1,5,5,7,8,8,8,7
223,1,B2,1,4,1,8,8,9,6,8,7,7,8,-3,7,8,7,7,8
234,1,B8,1,4,2,4,4,7,4,7,7,5,4,4,-1,0,8,7,-1


In [26]:
len(bank_df[(bank_df[bank_df.columns.drop(['Branch'])].astype(int).values < 0)])

65

In [27]:
(bank_df[bank_df.columns.drop(['Branch'])].astype(int).values < 0)

array([[False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [28]:
bank_df[bank_df['Pleasant Ambiance'].str.match('^-\d') == True]

,Response,Branch,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place
Sr. No,,,,,,,,,,,,,,,,,,,
20,0,B1,2,2,1,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3


In [29]:
bank_df[bank_df['Comfortable seating arrangement'].str.match('\d') == False]

,Response,Branch,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place
Sr. No,,,,,,,,,,,,,,,,,,,
20,0,B1,2,2,1,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3


##### Dropping rows with Non-response

In [30]:
bank_df.drop(bank_df[(bank_df[bank_df.columns.drop(['Branch'])].astype(int).values < 0)].index,inplace=True)

In [31]:
bank_df.reset_index(inplace=True)

In [32]:
bank_df.head()

,Sr. No,Response,Branch,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place
0,2,0,B1,1,5,1,10,8,8,6,10,8,9,6,8,6,8,7,10,10
1,3,0,B1,0,5,1,4,8,7,8,5,7,4,5,7,6,8,7,9,6
2,4,0,B1,6,4,2,4,6,6,8,6,4,4,5,6,6,9,6,9,5
3,5,0,B1,1,2,1,10,8,6,10,8,8,5,7,8,7,10,10,5,8
4,6,0,B1,2,3,1,7,8,6,7,8,7,7,7,6,7,8,7,8,7


In [33]:
bank_df.drop(columns=['Sr. No'],inplace=True)

In [34]:
bank_df.head(1)

,Response,Branch,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place
0,0,B1,1,5,1,10,8,8,6,10,8,9,6,8,6,8,7,10,10


In [35]:
bank_df['Branch'].unique()

array(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10'],
      dtype=object)

In [36]:
len(bank_df)

204

In [44]:
bank_df = bank_df.drop(columns=['Branch']).join(pd.get_dummies(bank_df['Branch'],dtype=int))

In [45]:
bank_df.head()

,Response,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place,B1,B10,B2,B3,B4,B5,B6,B7,B8,B9
0,0,1,5,1,10,8,8,6,10,8,9,6,8,6,8,7,10,10,1,0,0,0,0,0,0,0,0,0
1,0,0,5,1,4,8,7,8,5,7,4,5,7,6,8,7,9,6,1,0,0,0,0,0,0,0,0,0
2,0,6,4,2,4,6,6,8,6,4,4,5,6,6,9,6,9,5,1,0,0,0,0,0,0,0,0,0
3,0,1,2,1,10,8,6,10,8,8,5,7,8,7,10,10,5,8,1,0,0,0,0,0,0,0,0,0
4,0,2,3,1,7,8,6,7,8,7,7,7,6,7,8,7,8,7,1,0,0,0,0,0,0,0,0,0


In [46]:
bank_df.tail()

,Response,Occupation,Age,Gender,Pleasant Ambiance,Comfortable seating arrangement,Immediate attenttion,Good Response on Phone,Errors in Passbook entries,Time to issue cheque book,Time to sanction loan,Time to clear outstation cheques,Issue of clean currency notes,Facility to pay bills,Distance to residence,Distance to workplace,Courteous staff behaviour,Enough parking place,B1,B10,B2,B3,B4,B5,B6,B7,B8,B9
199,1,0,5,1,10,6,10,9,5,9,5,3,4,8,8,5,5,4,0,0,0,0,0,0,1,0,0,0
200,1,0,4,1,2,4,2,0,2,6,1,1,4,5,6,4,6,1,0,0,0,0,0,0,1,0,0,0
201,1,2,2,1,10,10,9,10,9,10,9,10,8,10,10,9,10,9,0,0,0,0,0,0,1,0,0,0
202,1,1,3,1,9,8,9,9,4,8,6,7,8,9,10,9,9,8,0,0,0,0,0,0,1,0,0,0
203,1,1,3,1,8,8,5,6,9,9,8,9,7,7,6,6,10,9,0,0,0,0,0,0,1,0,0,0


In [47]:
bank_df.columns

Index(['Response', 'Occupation', 'Age', 'Gender', 'Pleasant Ambiance',
       'Comfortable seating arrangement', 'Immediate attenttion',
       'Good Response on Phone', 'Errors in Passbook entries',
       'Time to issue cheque book', 'Time to sanction loan',
       'Time to clear outstation cheques', 'Issue of clean currency notes',
       'Facility to pay bills', 'Distance to residence',
       'Distance to workplace', 'Courteous staff behaviour',
       'Enough parking place', 'B1', 'B10', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7',
       'B8', 'B9'],
      dtype='object')

In [51]:
bank_df = bank_df.astype(int)

In [76]:
cat_var_lst = ['Pleasant Ambiance',
       'Comfortable seating arrangement', 'Immediate attenttion',
       'Good Response on Phone', 'Errors in Passbook entries',
       'Time to issue cheque book', 'Time to sanction loan',
       'Time to clear outstation cheques', 'Issue of clean currency notes',
       'Facility to pay bills', 'Distance to residence',
       'Distance to workplace', 'Courteous staff behaviour',
       'Enough parking place']

In [78]:
for i in cat_var_lst:
    bank_df[i] = bank_df[i].astype('category')

In [52]:
model = OrderedModel(bank_df['Response'],bank_df[['Pleasant Ambiance',
       'Comfortable seating arrangement', 'Immediate attenttion',
       'Good Response on Phone', 'Errors in Passbook entries',
       'Time to issue cheque book', 'Time to sanction loan',
       'Time to clear outstation cheques', 'Issue of clean currency notes',
       'Facility to pay bills', 'Distance to residence',
       'Distance to workplace', 'Courteous staff behaviour',
       'Enough parking place']],distr='probit')

In [53]:
model_prob = model.fit(method='bfgs')

Optimization terminated successfully.
         Current function value: 0.324796
         Iterations: 19
         Function evaluations: 23
         Gradient evaluations: 23


In [54]:
model_prob.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:               Response   Log-Likelihood:                -66.258
Model:                   OrderedModel   AIC:                             162.5
Method:            Maximum Likelihood   BIC:                             212.3
Date:                Tue, 26 Mar 2024                                         
Time:                        19:00:36                                         
No. Observations:                 204                                         
Df Residuals:                     189                                         
Df Model:                          14                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Pleasant Ambiance                    0.1966      0.087      2.250      0.024       0.025       0.368
Comfortable seating arrangement      0.0277      0.080      0.349      0.727      -0.128       0.184
Immediate attenttion                 0.0225      0.071      0.318      0.751      -0.116       0.161
Good Response on Phone              -0.0995      0.060     -1.666      0.096      -0.217       0.018
Errors in Passbook entries           0.0947      0.081      1.165      0.244      -0.065       0.254
Time to issue cheque book           -0.0621      0.072     -0.860      0.390      -0.204       0.079
Time to sanction loan               -0.0704      0.076     -0.922      0.357      -0.220       0.079
Time to clear outstation cheques     0.0765      0.076      1.001      0.317      -0.073       0.226
Issue of clean currency notes       -0.2079      0.089     -2.338      0.019      -0.382      -0.034
Facility to pay bills                0.1270      0.094      1.358      0.175      -0.056       0.310
Distance to residence               -0.0900      0.076     -1.186      0.236      -0.239       0.059
Distance to workplace               -0.1420      0.093     -1.533      0.125      -0.324       0.040
Courteous staff behaviour            0.0086      0.080      0.108      0.914      -0.147       0.165
Enough parking place                -0.0060      0.092     -0.065      0.948      -0.187       0.175
0/1                                  0.3994      0.690      0.579      0.563      -0.953       1.752
====================================================================================================
"""

In [79]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Response                          204 non-null    int64   
 1   Occupation                        204 non-null    int64   
 2   Age                               204 non-null    int64   
 3   Gender                            204 non-null    int64   
 4   Pleasant Ambiance                 204 non-null    category
 5   Comfortable seating arrangement   204 non-null    category
 6   Immediate attenttion              204 non-null    category
 7   Good Response on Phone            204 non-null    category
 8   Errors in Passbook entries        204 non-null    category
 9   Time to issue cheque book         204 non-null    category
 10  Time to sanction loan             204 non-null    category
 11  Time to clear outstation cheques  204 non-null    category

In [57]:
x_train, x_test, y_train, y_test = train_test_split(bank_df.drop(columns=['Response']),bank_df['Response'],test_size=0.25)

In [60]:
model_log = LogisticRegression(max_iter=1000)

In [62]:
model_log = model_log.fit(x_train,y_train)

In [63]:
model_log

LogisticRegression(max_iter=1000)

In [66]:
train_pred = model_log.predict(x_train)

In [67]:
confusion_matrix(y_train,train_pred)

array([[131,   2],
       [ 12,   8]])

In [68]:
test_pred = model_log.predict(x_test)

In [69]:
confusion_matrix(y_test,test_pred)

array([[38,  5],
       [ 6,  2]])

In [75]:
print(classification_report(y_test,test_pred))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87        43
           1       0.29      0.25      0.27         8

    accuracy                           0.78        51
   macro avg       0.57      0.57      0.57        51
weighted avg       0.77      0.78      0.78        51

